In [88]:
from datetime import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import time

In [89]:
#Simple dataclass for each Holiday object
@dataclass
class Holiday:  
    name: str
    date: str
    def __str__ (self):
        return (f'{self.name}({self.date})')

In [115]:
#Complex dataclass for a list of holiday objects, that also contains a 
#variety of functions for the holiday manager
@dataclass
class HolidayList:
    innerHolidays: list
    def __str__(self):
        
        return str(self.innerHolidays)
        
    def listlength(self):
        return len(self.innerHolidays)
    
    def addHoliday(self, holidayObj):
        if str(type(holidayObj)) == "<class '__main__.Holiday'>":
            self.innerHolidays.append(holidayObj)
        else:
            print('Sorry, that is not the correct object type.')

    def findHoliday(self, HolidayName, Date):
        for i in self.innerHolidays:
                if i.name == HolidayName and i.date == Date:
                    return i
        return False

    def removeHoliday(self, HolidayName, Date):
        for i in self.innerHolidays:
                if i.name == HolidayName and i.date == Date:
                    self.innerHolidays.remove(i)
                    print(f'{HolidayName} has been removed')

    def read_json(self,filelocation):
        with open(filelocation, "r") as data_file:
            data = json.loads(data_file.read())
            for dic in data["holidays"]:
                self.addHoliday(Holiday(dic["name"], dic["date"]))

    def save_to_json(self,filelocation):
        holidaydiclist = []
        for dic in self.innerHolidays:
            holidaydiclist.append({"name":dic.name, "date":dic.date})
        holidayJson = {"holidays": holidaydiclist}
        with open(filelocation, 'w') as fout:
            json.dump(holidayJson, fout)
        
    def scrapeHolidays(self):
        scrapedholidays = []
        try:            
            for year in range(2020,2025):
                html = requests.get(f"https://www.timeanddate.com/holidays/us/{year}").text
                soup = BeautifulSoup(html, 'html.parser')
                table = soup.find('table',attrs = {'id':'holidays-table'})

                for row in table.find_all('tr'):
                    namecells = row.select('td a')
                    holiday = {}
                    if namecells != []:
                        holiday['Holiday Name'] = namecells[0].string
                        if 'data-date' in str(row):
                            holiday['Date'] = datetime.utcfromtimestamp((int(row['data-date']))/1000).strftime('%Y-%m-%d')
                            scrapedholidays.append(holiday)
            for dic in scrapedholidays:
                holi = Holiday(dic['Holiday Name'], dic['Date'])
                if self.findHoliday(dic['Holiday Name'], dic['Date']) == False:
                        self.addHoliday(holi)
                
        except Exception as e:
            print(e)
    
    def filter_holidays_by_week(self, year, week_number):
        holidayList = list(filter(lambda row: datetime.strptime(row.date, '%Y-%m-%d').isocalendar()[0] == int(year)
                                    and datetime.strptime(row.date, '%Y-%m-%d').isocalendar()[1] == int(week_number), self.innerHolidays))
        return holidayList


    def displayHolidaysInWeek(self, holidayList):
        for i in holidayList:
            print(i)
            
    def getWeather(self):
        url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"

        querystring = {"q":"boulder,us","cnt":"10","units":"metric or imperial"}

        headers = {
            "X-RapidAPI-Key": "4706c42ea8msh652d9724b862623p1f636bjsn634f12b59182",
            "X-RapidAPI-Host": "community-open-weather-map.p.rapidapi.com"
        }
        try:
            response = requests.request("GET", url, headers=headers, params=querystring)
        except:
            print("Could not get weather from API")
        response.json()
        weather = response.json()
        weatherlist = []
        for i in range(0,7):
            weatherdic = {}
            weatherdic['weatherdate'] = datetime.utcfromtimestamp(weather['list'][i]['dt']).strftime('%Y-%m-%d')
            weatherdic['weather']= weather['list'][i]['weather'][0]['description']
            weatherlist.append(weatherdic)
        return weatherlist
         # Convert weekNum to range between two days
         # Use Try / Except to catch problems
         # Query API for weather in that week range
         # Format weather information and return weather string.

    def viewCurrentWeek(self,HolidayList):
        weatherlist = self.getWeather()
        for index, value in enumerate(HolidayList):
            for holiday in weatherlist:
                if HolidayList[index].date == holiday['weatherdate']:
                    print(f"{value} {holiday['weather']}")

In [118]:
def main():
    #initializing HolidayList Object
    hl = HolidayList([])
    #Setting save variable
    unsaved = True
    #variable for exiting the menu
    end = False
    #read in starter json
    hl.read_json('holidays.json')
    #scrape holidays
    hl.scrapeHolidays()
    #find length of inner holiday list
    listlength = hl.listlength()
    print('\n')

    #print startup message
    
    print(f'''        Holiday Management
        ===============
        There are {listlength} holidays in the system\n\n''')

    #menu loop
    while end == False:
        print('''        Holiday Menu
        ===============
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiay List
        4. View Holidays
        5. Exit\n''')
        # For whatever reason, the print statement was sometimes appearing
        #after the user input, and I found that a small sleep time fixes it
        time.sleep(0.03)
        mnum = int(input('Enter a number (1-5) to choose a menu item: '))
    #Add a holiday
        if mnum == 1:
            while True:
                print('Add a Holiday')
                print('================')

                uhname = input('Holiday Name: ')
                uhdate = input(f'Holiday Date (yyyy-mm-dd): ')
                if len(uhdate) == 10:
                    hl.addHoliday(Holiday(uhname, uhdate))
                    print(f'{uhname} ({uhdate}) has been added to the holiday list.')
                    unsaved = True
                    break
                else:
                    print('That was not a valid date')
   #remove a holiday
        if mnum == 2:
             while True:
                print('Remove a Holiday')
                print('=================')

                uhrname = input('Holiday Name: ')
                uhrdate = input('Holiday Date: ')
                if hl.findHoliday(uhrname, uhrdate) == False:
                    print('Error')
                    print(f'{uhrdate} not found.')
                else:
                    hl.removeHoliday(uhrname, uhrdate)
                    unsaved = True
                    break
    #Save Holiday List    
        if mnum == 3:
            while True:
                print('Saving Holiday List')
                print('===================')

                save = input('Are you sure you want to save your changes? [y/n]')
                if save.lower() == 'n':
                    print('Canceled:')
                    print('Holiday list file save canceled')
                    break
                elif save.lower() == 'y':
                    hl.save_to_json('HolidaySaveFile')
                    unsaved = False
                    print('Your file has been saved.')
                    break
                else:
                    print('That is not a valid option')
    #view holidays filtered on year and week
        elif mnum == 4:
            print('View Holidays')
            print('==============')
            vyear = input('Which year?: ')
            vweek = input('Which Week?: #[1-52, leave blank for current week]')
            if vweek == '':
                vweek = datetime.now().isocalendar()[1]
                weather = input('Would you like to see the weather?[y/n]: ')
                print('\n')
                if weather.lower() == 'y':
                    print('These are the holiays for this week (and weather): ')
                    hl.viewCurrentWeek(hl.filter_holidays_by_week(vyear,vweek))
                else:
                    print('These are the holiays for this week: ')
                    hl.displayHolidaysInWeek(hl.filter_holidays_by_week(vyear,vweek))
            else:
                hl.displayHolidaysInWeek(hl.filter_holidays_by_week(vyear,vweek))
    #exit the holiday manager, different prompts based on whether or not
    #there are unsaved changes
        elif mnum == 5:
            if unsaved == False:
                print('Exit')
                print('=====')
                userexits = input('Are you sure you want to exit? [y/n]')
            
                if userexits == 'y':
                    end = True
            if unsaved == True:
                print('Exit')
                print('=====')
                userexituns = input('Are you sure you want to exit?\nYour changes will be lost.[y/n]')
                if userexituns == 'y':
                    print('Goodbye!')
                    end = True
    # Large Pseudo Code steps
    # -------------------------------------
    # 1. Initialize HolidayList Object
    # 2. Load JSON file via HolidayList read_json function
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    # 3. Create while loop for user to keep adding or working with the Calender
    # 4. Display User Menu (Print the menu)
    # 5. Take user input for their action based on Menu and check the user input for errors
    # 6. Run appropriate method from the HolidayList object depending on what the user input is
    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 


if __name__ == "__main__":
    main();



        Holiday Management
        There are 2630 holidays in the system


        Holiday Menu
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiay List
        4. View Holidays
        5. Exit

Enter a number (1-5) to choose a menu item: 4
View Holidays
Which year?: 2022
Which Week?: #[1-52, leave blank for current week]
Would you like to see the weather?[y/n]: y


These are the holiays for this week (and weather): 
World Population Day(2022-07-11) light rain
Nathan Bedford Forrest Day(2022-07-13) light rain
Bastille Day(2022-07-14) light rain
World Youth Skills Day(2022-07-15) light rain
Rural Transit Day(2022-07-16) light rain
        Holiday Menu
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiay List
        4. View Holidays
        5. Exit

Enter a number (1-5) to choose a menu item: 5
Exit
=====
Are you sure you want to exit?
Your changes will be lost.[y/n]y
Goodbye!
